# Sentence Generation
> 참고 : https://terabox.com/s/1BfWuiOGhgZEuvjOqnUbMew

```
작사하는 모델은 자연어 task 중 Sentence Generation에 속합니다.
그래서 영어 데이터가 아닌 한국어 데이터로 더 좋은 성능의 
딥러닝 모델(GPT)를 통해 문장 생성이 어떻게 이루어지는지 확인해보겠습니다.
```

**Sentence Generation**은 이전 Context(문맥)에서 다음 단어가 어떤 단어가 오는지 적절하게 분류하는 것입니다. 문장 생성에서 모델의 input은 Context가 되며, 출력은 context의 다음 토큰의 등장 확률이 됩니다.

- 다음은 입력으로 "점심"을 Context로 입력 받을 때 모델의 입출력입니다.
  - 입력 : 점심
  - 출력 :
    - 가 : 0.002
    - 가격 : 0.00001
    - ....
    - 나 : 0.005
    - 나눔 : 0.0005
    - ....
    - 다 : 0.004
    - 드: 0.2
    - 드셨나요: 0.5
    - ...
    - !: 0.15
    - ?: 0.18

아래 그림은 위 입출력을 도식화 한것입니다.
![IMAGE](https://drive.google.com/uc?id=1h8F5OhFud0SrY9K4FhsfDJc8nS02iCiE)


Sentence Generation의 방식은 다음과 같습니다. 
1. P(W|Context) 를 출력하게 한 뒤 다음 토큰을 선택합니다.
2. 기존 Context에 1 에서 선택한 다음 토큰을 이어 붙인 새로운 Context를 모델이 입력해서 다음 토큰 확률분포, 즉 P(W|New Context)를 추출하고 그 다음 토큰을 선택합니다.
3. 이를 반복해 다음 토큰을 계속 생성합니다.
4. 이렇게 생성한 토큰을 Post Processing을 통해 사람이 보기 좋은 형태로 가공합니다.


이번 Sentence Generation task 에서는 NSMC(Naver Sentiment Movie Corpus) 데이터를 활용해, SK텔레콤이 공개한 KoGPT2 모델을 파인튜닝하는 실습을 진행 할것입니다.

# 모델 구조

이번 Task에서 사용할 모델은 언어모델(Language Model) 입니다. Context가 주어졌을 때 다음 단어를 맞추는 방식으로 pretrain을 수행한 모델입니다. 대표적인 모델은 GPT, BERT가 있습니다.

![IMAGE](https://drive.google.com/uc?id=1SODQs2wIp501mMIkhkj0FguNMDAqSKk6)



---

# 환경 설정 및 모델 파인 튜닝하기

In [ ]:
!pip install ratsnlp # TPU이외의 의존성있는 패키지 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 845 kB/s 
     |████████████████████████████████| 2.8 MB 8.0 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 582 kB 70.0 MB/s 
     |████████████████████████████████| 512 kB 72.4 MB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
     |████████████████████████████████| 182 kB 64.4 MB/s 
     |████████████████████████████████| 880 kB 55.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d9e7e608703c75263e56e98a1526fe7728e5c08eb76d844f1edb57fa7694677c
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


### 구글 드라이브 연결

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


### 모델 환경 설정

**GenerationTrainArguments**

- `pretrained_model_name` : pretrain 마친 언어모델의 이름( 단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야함), 이번 task에서는 sk텔레콤에서 공개한 KoGPT2를 사용합니다.

- `downstream_corpus_name` : 다운 스트림 데이터의 이름

- `downstream_mdoel_dir` : 파인튜닝된 모델의 체크포인트가 저장될 위치

- `max_seq_length` : 토큰 기준 입력 문장 최대길이. 이보다 긴 문장은 자르고 짧은 문장은 같은 길이가 되도록 뒤에 padding 합니다.

- `batch_size` : 배치 사이즈, GPU 가속을 선택했다면 32 , TPU 가속이라면 4, 코랩 환경은 보통 TPU 8개 할당, 코어별로 적용되게끔 설정했습니다.

- `learing_rate` : 1회 스텝에서 한번에 얼마나 업데이트 할지에 관한 크기를 지정합니다.

- `epochs` : 학습 에포크 수 

- `tpu_cores` : TPU코어수, GPU가속을 선택했다면 0, TPU라면 8로 지정

- `seed` : 랜덤 시드(정수)

In [ ]:
# 모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name='skt/kogpt2-base-v2',
    downstream_corpus_name ="nsmc",
    downstream_model_dir = '/gdrive/My Drive/nlpbook/checkpoint-generation',
    max_seq_length = 32,
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    epochs = 3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed= 7
)
print("setting is DONE!!")

setting is DONE!!


### Random Seed 고정

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


### Log 출력

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cp

## Corpus 내려받기
Korpora는 데이터를 받는 오픈소스 라이브러리입니다. corpus_name에 해당하는 말뭉치를 코랩 환경 로컬  root_dir 이하에 저장합니다.


### Data Description
Each file is consisted of three columns: id, document, label
- `id`: The review id, provieded by Naver
- `document`: The actual review
- `label`: The sentiment class of the review. (0: negative, 1: positive)
Columns are delimited with tabs (i.e., .tsv format; but the file extension is .txt for easy access for novices)

### Qucik Peek
```
id      document        label

9976970 아 더빙.. 진짜 짜증나네요 목소리        0
3819312 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나        1
10265843        너무재밓었다그래서보는것을추천한다      0
9045019 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정       0
6483659 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다  1
5403919 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.     0
7797314 원작의 긴장감을 제대로 살려내지못했다.  0
9443947 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네       0
7156791 액션이 없는데도 재미 있는 몇안되는 영화 1
```

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download = args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 273MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 154MB/s]


## 데이터 전처리하기
딥러닝 모델을 학습하려면 학습데이터를 배치 단위로 지속적으로 모델에 공급해 주어야 합니다. pytorch에서는 이 역할을 데이터 로더(DataLoader)가 수행해줍니다.

NSMC를 다음과 같이 처리합니다. 파인튜닝 시 문장 맨앞에 극성(polarity) 정보를 부여함으로써, 인퍼런스 과정에서 임의의 극성 정보를 주었을 때 해당 극성에 맞는 문장을 생성할 수 있는 능력이 있는지 여부를 검증해보고자 합니다.

```
부정 아 더빙.. 진짜 짜증나네요 목소리
부정 흠...포스터보고 초딩영화인줄...
긍정 너무재밓었다그래서보는것을추천한다
(하약)
```

`GenerationDataset` 은 `NsmcCorpus`가 넘겨준 문장으로 `input_ids`, `attention_mask`, `token_type_ids`, `labels` 네 자료로 변환하는 역할을 합니다.

위 `부정 아 더빙.. 진짜 짜증나네요 목소리` 문장을 통해 tokenizer를 한 결과 입니다.

- **Input_ids** : [11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

- **attention_mask**:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

- **token_type_ids**: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

- **labels** : [11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


`Input_ids`에서 1이 많은 이유는 문장 뒤에 패딩토큰(`</s>`)가 붙어 있어서 입니다. 이 토큰이 붙은 이유는 모델환경설정에서 `max_seq_length`보다 짧아서 입니다. 

`token_type_ids`는 세그먼트 정보로 기본값은 모두 0으로 넣습니다.

### Tokenizer 준비
SK텔레콤이 공개한 KoGPT2 모델이 사용하는 토크나이저를 선언합니다. `eos_token`은 문장 마지막에 붙이는 스페셜 토큰입니다.(end of sentence) pretrain시 이렇게 지정했기 때문에 같은 방식을 사용합니다.

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# 학습데이터 구축
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train"
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 7.108 s]
INFO:ratsnlp:tokenize sentences [took 7.108 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> <

In [ ]:
# 학습데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement=False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers=args.cpu_workers
)

- `sampler` : 배치를 만들때 `GenerationDataset`이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼 비복원(`replacement=False`)랜덤 추출합니다.
- `collate_fn` : `sampler`로 뽑힌 인스턴스를 배치로 반드는 역할을 하는 함수입니다. `GenerationDataset` 은 list형태의 자료형인데 파이토치에서 요구하는 tensor 형태로 바꾸는 역할을 수행합니다.

In [ ]:
# 평가용 데이터로더 구축
from torch.utils.data import SequentialSampler
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='test',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 2.522 s]
INFO:ratsnlp:tokenize sentences [took 2.522 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


## 모델 빌딩

In [ ]:
# 모델 초기화
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

## 모델 파인튜닝하기

- pytorch ligthning이 제공하는 라이트닝 모듈을 상속받아 task를 지정합니다.
- 모델은 모델 초기화에서 선언해준 모델을 사용합니다. 옵티마이저는 웜업 스케줄링을 적용한 Adam을 사용합니다.
- Trainer 정의를 통해 GPU/TPU설정, 로그 및 체크 포인트 등을 알아서 설정해줍니다.


In [ ]:
# task 정의
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [ ]:
# trainer 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# 학습 개시
trainer.fit(
    task,
    train_dataloader,
     val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# Pretrain을 마친 모델로 문장생성하기

In [ ]:
#체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)

model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
# 토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# 모델 입력값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

- 위 코드를 실행하면 KoGPT2 모델에 넣을 입력값(Context)를 만들 수 있습니다. 
- 토크나이저의 `encode` 메소드는 입력 문장(string)을 토큰화 한 뒤 정수로 인덱싱하는 역할을 수행합니다.
- `return_tensors` 인자를 "pt"로 주면 인덱싱 결과를 파이토치의 tensor 자료형으로 반환합니다.
- 아래에 `input_ids`를 확인해보면 tensor로 변환되어 있습니다.

In [ ]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

## Grid Search

- 그리드 서치는 매순간 최선의최선을 선택해 탐색 범위를 줄여보자는 것이 핵심 아이디어입니다.
- 언어 모델(Language Model)은 Context(단어 혹은 단어 시퀀스)를 입력 받아 다음 단어가 나타날 확률을 출력으로 반환합니다.
- 모델의 출력확률분포로부터 다음 토큰을 반복적으로 선택하는 과정이 바로 문장 생성 태크스가 됩니다.
- 하지만 문제는 특정 Context 다음에 올 단어는 무수히 많은 경우의 수가 존재한다는 것입니다. 

- 다음 그림은 그 예시 입니다. `그` 라는 Context로 부터 출발해 다음 단어로 어떤 것이 올지 언어모델이 예측한 결과 입니다.

![IMAGE](https://drive.google.com/uc?id=1usO1-1jMJmP6Z647WE1SfmeAEjOnpHBc)

- 위 그림을 통해 본 생성된 문장을 추려보면 다음과 같습니다.
```
그 집 사
그 집은
그 집에
그 책이
그 책을
그 책읽
그 사람에게
그 사람처럼
그 사람 누구
```
- 정석대로 문장을 생성하려면 저 9가지의 모든 케이스를 모델에 입력해서 다음 단어 확률분포를 계산하고 이로부터 다음 단어를 선택하는 과정을 거쳐야합니다. 이러한 과정을 계속해서 반복해야합니다.

- 단순히 9가지로 적었지만 실제로는 셀수 없을 만큼 많습니다. 그래서 모든 경우의 수를 계산하는 것은 불가능에 가깝습니다.

- 하지만 이러한 문제를 해결해주는 것이 그리디 서치입니다.

- 다음 그림은 그리드 서치를 적용한 것입니다.

![imgae](https://drive.google.com/uc?id=1-V2FS-RLQE8IZdz2RMMphFdn1jtNBtix)

- `그` 다음에 올 단어로 `책`이 0.5 로 가장 높다고 예측하였습니다.
- `책`을 선택하고 `그 책`을 모델에 입력해 다음 단어 확률분포를 계산합니다.
- 셋 가운데 확률이 0.4로 가장 높은 `이`를 선택합니다.
- 다음 코드는 그리디 서치를 구현한 코드입니다.  

In [ ]:
# 그리디 서치
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length = 10,
      max_length = 50,
  )

In [ ]:
generated_ids

tensor([[25906,  8702,  7801,  8084,   406, 16563,   377,  6947,  7401,   387,
          9022,  6855, 46651,  8046,  8084,   406, 16563,   377,  6947,  7401,
           387,  9022,  6855, 46651,  8046,  8084,   406, 16563,   377,  6947,
          7401,   387,  9022,  6855, 46651,  8046,  8084,   406, 16563,   377,
          6947,  7401,   387,  9022,  6855, 46651,  8046,  8084,   406, 16563]])

- 위 결과를 살펴보면 토큰ID라 사람이 알아보기 어렵습니다. 다음 코드를 통해 문장으로 변환해 줍니다 

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 빔 서치(Beam Search)

- 빔서치는 beam의 크기만큼 선택지를 계산 범위에 넣습니다.
- `그` 다음에 올 단어로 모델은 `책`(0.5), `집`(0.4), `사람`(0.1) 순으로 예측했습니다. 저희는 빔 크기를 2로 두었으므로 `사람`은 제거하고 `책`과 `집`만 탐색 대상으로 남겨 둡니다.

- 모델에 `그 책`을 입력해 단어 시퀸스 확률을 계산합니다.
  - `그 책이` :  $0.5 × 0.4 = 0.2$
  - `그 책을` : $0.5 \times 0.3 = 0.15$
  - `그 책읽` : $0.5 × 0.3 = 0.15$

- 모델에 `그 집`을 입력해 단어 시퀸스 확률을 계산합니다.
  - `그 집에` : $0.4 × 0.7 = 0.28$
  - `그 집은` : $0.4 × 0.2 = 0.08$
  - `그 집 사` : $0.4 × 0.1 = 0.04$

- 빔 크기가 2이므로 6가지 경우의 수에서 가장 높은 확률을 가진 시퀸스 2개 만 남겨둡니다. 
- `그 집에`(0.28), `그 책이`(0.2) 입니다.빔 서치를 그만 둔다면 여기서 가장 높은 `그 집에`를 최종 생성 결과가 됩니다.

- 아래 그림은 빔 서치를 나타낸 그림입니다.
![image](https://drive.google.com/uc?id=1W43Yau4gCl2N9BL108JR-eVDPkzwl_ab)


- 빔 서치는 그리디 서치 대비 계산량은 많은 편입니다. 그리디 서치는 매순간 최고의 확률을 선택하지만 빔 서치는 빔 크기만큼의 경우의 수를 계산하기 때문입니다. 
- 하지만 빔 서치는 그리디 서치보다 조금이라도 더 높은 확률을 내는 문장을 생성 할 수 있는 가능성을 높이게 됩니다.

- 다음 코드는 빔 서치를 구현한것입니다.

In [ ]:
#빔 서치
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length =  10,
      max_length = 50,
      num_beams = 3,
  )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


- 위 코드에서 중요한 인자는 `do_sample=False`, `num_beams=3` 입니다. 
- `num_beams`는 빔 크기를 가리킵니다.
- `num_beams=`로 설정한다면 그리드 서치로 동작하게 됩니다.

---

## 특정 단어 반복 줄이기

- 그리드 서치('그럼, 그건 뭐에요?')와 빔 서치("그렇지 않습니다")에서 특정 표현이 반복되는 것을 볼 수 있었습니다.
- 토큰이 n-gram 단위로 반복될 경우 모델이 계산한 결과를 무시하고, 해당 n-gram의 등장 확률을 0으로 만들어 생성해서 배제하게 됩니다.
- 핵심 인자 `no_repeat_ngram_size=3` 입니다.
- 3개 이상의 토큰이 반복될 경우 3-gram 등장확률을 인위적으로 0으로 만듭니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length = 10,
      max_length = 50,
      no_repeat_ngram_size = 3
  )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


- 위 결과를 보면 n-gram 등장 확률을 인위적으로 조정한 것 뿐입니다. 
- 최대 확률을 내는 단어 시퀀스를 찾는다는 본질이 바뀐게 아니므로 위 코드를 반복수행해도 결과는 똑같습니다.
- 다음은 `repetition_penalty` 라는 인자를 통해서 반복되는 단어들을 통제 해보겠습니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length = 10,
      max_length = 50,
      no_repeat_ngram_size = 3,
      repetition_penalty = 1.1,

  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요? 저는 지금 이 순간에도 괜찮아요."
"그래요, 그럼. 그리고 오늘은 제가 좋아하는 영화도 보고 싶습니다."
"아, 그래요? 그럼


In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length = 10,
      max_length = 50,
      no_repeat_ngram_size = 3,
      repetition_penalty = 1.2,

  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = False,
      min_length = 10,
      max_length = 50,
      no_repeat_ngram_size = 3,
      repetition_penalty = 1.5,

  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


## Top K 샘플링
```
- 지금까지의 방식은 모델이 출력한 다음 토큰 확률분포를 점수로 활용한것입니다.
- 전체 어휘 가운데 점수가 가장 높은 토큰을 다음 토큰으로 결정하는 방식입니다.
- 위 방식으로 진행하게 되면 동일한 모델에 동일한 context를 입력하는 경우 문장 생성을 여러번 반복해도 똑같습니다.
```
- 샘플링은 아래와 같은 이미지로 볼 수 있습니다.
![IMAGE](https://drive.google.com/uc?id=1juW3PA7dWJsU1u4sqbKCPQNsZf89xNl0) 

- `그` 라는 context를 입력했을 때 다음 토큰으로 `집`(0.5), `책`(0.4), `사람`(0.1) 으로 예측했습니다.
- 그리드 서치나 빔 서치의 방식을 택하면 시행 시 같은 토큰 `집`으로만 예측할 것입니다.
- 하지만 샘플링 방식을 통해 매 시행 시 다음에 오는 토큰이 달라 질 수 있습니다.


- 탑K 샘플링은 모델이 예측한 다음 토큰 확률 분포에서 확률값이 가장 높은 K개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법입니다.

- 다음 그림은 context를 `그`, k=5로 두었을때 샘플링 대상 토큰을 나타낸 것 입니다.
![IMGAE](https://drive.google.com/uc?id=1sAzkcZUa0P3xx8lcdrTtNhSatHc1zhy_)
- 탑k 샘플링을 구현해 보겠습니다.
- 핵심인자는 `do_sample` 과 `top_k` 입니다.

In [ ]:
# 탑K샘플링
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = True,
      min_length = 10,
      max_length = 50,
      top_k=50,

  )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"어머머머, 정말 예뻐요!"
"아, 아니에요. 오늘은 제가 한 건데."
소녀들은 그제서야 아까부터 소녀들의 이야기에 귀를 기울일 수 있었다.
그들은 소녀


## 템퍼러쳐 스케일링
- 모델의다음 토큰 확률분포에 변형을 가해 문장을 다양하게 생성하는 기법입니다.
- 확률분포를 변형한다는 의미는, 대소 관계의 역전없이 분포의 모양만 바꾼다는걸 말합니다.
- 다시말해, 높은 확률은 가진 토큰은 더 높게, 작았던 확률은 더 작게 변환하는 것입니다.
- 다음은 `temperature` 인자 설명입니다.
  - `temperature` 인자를 통해 템퍼러쳐 스케일링을 적용합니다.
  - 0 이상의 값을 가져야 하며 0에 가까울 수록 스켈일링이 더 강하게 됩니다.
  - 1로 설정한다면 모델이 출력한 확률분포를 어떤 변형없이 사용한다는 의미가 됩니다.
  - 1보다 큰 값을 둔다면 확률분포가 평평해집니다(uniform)
  - 높은 확률과 낮은 확률의 차이가 줄어든다는 이야기입니다.
  - 다양한 문장이 생성될 가능성이 높지만 생성 문장의 품질이 나빠질 수 있습니다.
- 정리하자면 `temperature` 를 1보다 작세 하면 상대적으로 정확한 문장을, 1보다 크게 하면 상개적으로 다양한 문장을 생성 할 수 있습니다.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요? 네. 뭐! 그럼 제가 드릴 말씀도 없을까? 혹시, 어서 오세요."
수철이 다가와서, 수철은 수철의 머리를 쓰다듬으며 말했다.
그의 눈은 수철의 얼굴과 어울려서 매우 따뜻하고 포근해


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요"
홍수는 내 등을 질썩 문근영으로 안긴 것은 다름바로 내가 이놈 저집 애들보다 늠나 좋아했다고 하자 문둥이란 얼굴 한 겹 보기로 이쪽 가슴을 감싸기 위하여 발길에 한 줄기


## 탑 P 샘플링
- 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적확률값이 $ p $ 이하인 단어들 가운데 하나를 다음 단어로 선택하는 기법입니다.
- 확률값 기준으로 단어들을 내림차순 정렬해 그 값이 높은 단어들을 후보로 삼는다는 점에서 탑k샘플링 방식과 같지만 상위 $ k $ 개를 후보로 삼느냐, 누적 확률값 $ p $이하인 단어들을 후보로 삼느냐에 따라 차이가 있습니다.
- 탑 p 샘플링은 누적 확률 합으로 후보 단어를 취하기 때문에 누적 확률합이 일정한 반면 후보 단어 갯수는 해당 분포에 따라 달라지게 됩니다. 
- 탑 k 샘플링은 단어 갯수로 후보 단어를 취하기 때문에 후보 단어 개수는 일정한 반면 누적 확률합은 해당 분포에 따라 달라집니다. 다만, 둘 모두 확률값이 낮은 단어는 다음 단어 후보에서 제거하기 때문에 높은 문장을 생성할 가능성을 높이게 됩니다.

- 다음은 탑 p 샘플링을 구현한 코드 입니다.
  - `do_sample`과 `top_p`가 핵심인자입니다.
  - `do_sample=True`는 샘플링 방식을 취하겠다는 뜻이며 `top_p=0.92`는 $ p =0.92 $ 로 설정 하겠다는 뜻입니다.
  - `top_p`는  0 ~ 1  사이 값입니다.
  - 0에 가까워질 수록 후보 단어 수가 줄어들어 그리드 서치와 비슷해집니다.
  - 1로 설정한다면 확률값이 낮은 단어를 전혀 배제하지 않고 다음 단어 후보로 전체 어휘를 고려한다는 의미가 됩니다.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"어머님. 우리 식구들이 좀 다 컸어요."
"아이들, 이제야 아빠가 안심이 되셨어요."
"걱정마세요, 아빠. 빨리 와요."
그


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.80,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"라는 질문을 받았다.
그녀는 "제가 좀 피곤한 것 같아서 그러는 거죠. 오늘은 너무 피곤해서 그러는 거죠"라고 말했다.
그러나 그녀는 "아무래도 오늘은 너무 피곤해요"라고 말하면서 "제가


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"하고 말을 해야 한다.
그러면 그는 갑자기 자기가 좋아하는 일을 하는 것이 아닌가?
"이것은 뭐예요?"하고 대답해야 한다.
그는 이렇게 말한다.
"당신이 당신의 직업을 결정하는 데 어떻게 도움을 줘요?"


In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.02,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 종합적용하기
- 지금까지 설명했던 문장 생성 방식을 모두 적용해보겠습니다.

In [ ]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?' 하는 질문이 나오는 순간부터 '왜 자꾸 그러느냐, 이런 식으로 반응한다'는 답변이 이어진다.
아마도 이러한 태도는 자신을 향해 끊임없이 공격을 하고 있는 기자들이기 때문일 것이다.
그러던 중 어느 날 문득 김


In [10]:
# 모델 입력값 만들기
text = ["안녕하세요", "날 버리지마", "사랑해", "거울 속에"]
for i in range(len(text)):
  input_ids = tokenizer.encode(text[i], return_tensors="pt")

  with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))


안녕하세요~ 짱!ᄒ재미있어</s>
날 버리지마요 ᅲ_ᅲ 잘<unk>음</s>
사랑해피투게더. ^^ 강추</s>
거울 속에 남는건 노래뿐.. 모든게 좋았다</s>


In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 14.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=417c9c9281c986bc3885d45d7fd2e108c54592e03cabbcf95777dc9907dc7b94
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

In [15]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


^ 런타임 유형이 변경되었을 때만 실행


In [16]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


### 토크나이저 및 모델 불러오기

In [17]:
# 토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
from transformers.utils.dummy_pt_objects import PretrainedBartModel
# 모델 로드
import torch
from transformers import GPT2Config, GPT2LMHeadModel
if args.downstream_model_checkpoint_fpath is None:
  model = GPT2LMHeadModel.from_pretrained(
      args.pretrained_model_name,
  )
else:
  from google.colab import drive
  drive.mount('/gdrive', force_remount=True)
  pretrained_model_config = GPT2Config.from_pretrained(
      args.pretrained_model_name,
  )
  model = GPT2LMHeadModel(pretrained_model_config)
  fine_tuned_model_ckpt = torch.load(
      args.downstream_model_checkpoint_fpath,
      map_location=torch.device("cpu")
      )
  
  model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Mounted at /gdrive


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


### 모델 출력값 만들고 후처리하기

In [19]:
from IPython.core.interactiveshell import re
from torch.nn.modules.module import T
def inference_fn(
    prompt,
    min_length=10,
    max_length=50,
    top_p=1.0,
    top_k=50,
    repetition_penalty=1.0,
    no_repeat_ngram_size=0,
    temperature=1.0,):
  try:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
      generated_ids = model.generate(
          input_ids,
          do_sample=True,
          top_p=float(top_p),
          top_k=int(top_k),
          min_length=int(min_length),
          max_length=int(max_length),
          repetition_penalty=float(repetition_penalty),
          no_repeat_ngram_size=int(no_repeat_ngram_size),
      )
    generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
  except:
    generated_sentence = """ 처리 중 오류가 발생했습니다. <br>
    변수의 입력 범위를 확인하세요. <br><br>
    min_length : 1 이상의 정수 <br>
    max_length : 1 이상의 정수 <br>
    top_p : 0 ~ 1 사이의 실수 <br>
    top_k : 1 이상의 정수 <\br>
    repetition_penalty : 1 이상의 실수 <br>
    no_repeat_ngram_size : 1 이상의 정수 <br>
    temperature : 0 이상의 실수
    """
  return {
      'result': generated_sentence
  }


In [26]:
text = ['부정 이거 ', '긍정 이리도', '부정 아.', '긍정 오잉?','긍정 아 정말',  '부정 하지만', '긍정 이거']
for i in range(len(text)):
  temp = inference_fn(text[i])
  print(temp)

{'result': '부정 이거 뭔 생각으로 본지..참,.시간 아까운 영화였다;;;;</s>'}
{'result': '긍정 이리도 재미있는영화를 찾기 어려운듯. 영화보는내내 너무 재미있었다.</s>'}
{'result': '부정 아.,.,.,.,;;</s>'}
{'result': '긍정 오잉?ᄏ... 진짜 대박임..</s>'}
{'result': '긍정 아 정말 재밋게 잘 <unk>던 영화임</s>'}
{'result': '부정 하지만 좀 더 디어나가진 못했지만 그래도 배우들의 연기나 배우들의 연기력은 좋음</s>'}
{'result': '긍정 이거보고 암이 나았습니다</s>'}


# 결론 및 회고

- 한국어임에도 불구하고 loss 는 2.29로 어느 정도 만족한 수치로 나온것 같습니다.
- 문장을 보니 어느 정도 잘 생성 되는 것 같습니다.
- pytorch를 이용해 Pre train된 GPT2모델을 사용해 보았는데 확실히 tensorflow보다 구축이 훨씬더 쉽다는 느낌을 받았습니다.
- 출력 문장을 조금 더 정제하여 평서문처럼 보이게 할 필요가 있습니다. 
- 지금은 NSMC 데이터를 사용하였지만 cutomize data에서도 잘 작동되는지 확인해 봐야겠습니다.

```
Pre Traind : 기존에 자비어(Xavier) 등 임의의 값으로 초기화하던 모델의 가중치들을 다른 문제(task)에 학습시킨 가중치들로 초기화하는 방법이다.

예를 들어, 텍스트 유사도 예측 모델을 만들기 전 감정 분석 문제를 학습한 모델의 가중치를 활용해 텍스트 유사도 모델의 가중치로 활용하는 방법이다.
즉, 감정 분석 문제를 학습하면서 얻은 언어에 대한 이해를 학습한 후 그 정보를 유사도 문제를 학습하는 데 활용하는 방식이다.
이때 사전 학습한 가중치를 활용해 학습하고자 하는 본 문제를 하위 문제(downstream task) 라 한다. 
앞 예시에서 사전 학습한 모델인 감정 분석 문제가 사전 학습 문제(pre-train task) 가 된
다.
```
```
Fine tuning :  사전 학습한 모든 가중치와 더불어 downstream task를 위한 최소한의 가중치를 추가해서 모델을 추가로 학습(미세 조정) 하는 방법이다.

앞 예시의 사전 학습 방법인 감정 분석 문제에 사전 학습시킨 가중치와 더불어 텍스트 유사도를 위한 부가적인 가중치를 추가해 텍스트 유사도 문제를 학습하는 것이 미세 조정 방법이다.
```